In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

        
X = pd.read_csv("train.csv",index_col=0)
X_test = pd.read_csv("test.csv", index_col=0)



OHE and fix NaN values and exclude columns with more than 50% of NaN values.

In [22]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

print(X.shape)
print(X_test.shape)

X_full = pd.concat([X,X_test],axis=0)

# print(X_full.shape)

# print(X_full.info())

n_col = [colname for colname in X_full.columns if X_full[colname].dtype in ["int64","float64"]]
cate_col = [colname for colname in X_full.columns if X_full[colname].dtype == "object"]
print(len(cate_col) + len(n_col))

imputer = SimpleImputer(strategy="mean")
imputer2 = SimpleImputer(strategy="most_frequent")
OH = OneHotEncoder(sparse=False,handle_unknown="ignore")

# print(X_full.shape)
X_full = pd.concat([pd.DataFrame(imputer.fit_transform(X_full[n_col]),index=X_full.index,columns=X_full[n_col].columns),X_full[cate_col]],axis=1)


#Delete columns with > 50% NaN values
X_full.drop(["PoolQC","MiscFeature","Alley","Fence","FireplaceQu"],inplace=True,axis=1)

#Get Category columns
cate_col = [colname for colname in X_full.columns if X_full[colname].dtype == "object"]

X_full = pd.concat([pd.DataFrame(imputer2.fit_transform(X_full[cate_col]),index=X_full.index,columns=X_full[cate_col].columns),X_full[n_col]],axis=1)

OH_col = pd.DataFrame(OH.fit_transform(X_full[cate_col]),index=X_full.index)

X_full = pd.concat([X_full,OH_col],axis=1)

X_full.drop(cate_col,inplace=True,axis=1)

print(X_full.isnull().sum().sort_values(ascending=False))
print(X_full.shape)

X_clean = X_full.iloc[:1460]
X_test_clean = X_full.iloc[1460:]
y = X_clean["SalePrice"]
X_clean.drop("SalePrice",axis=1,inplace=True)
X_test_clean.drop("SalePrice",axis=1,inplace=True)

print(X_clean.shape)
print(X_test_clean.shape)


(1460, 80)
(1459, 79)
80
MSSubClass    0
149           0
135           0
136           0
137           0
             ..
59            0
60            0
61            0
62            0
233           0
Length: 271, dtype: int64
(2919, 271)
(1460, 270)
(1459, 270)


Feature Selection

In [34]:
from matplotlib import pyplot as ptl
import seaborn as snb
correlation_table = pd.concat([X_clean,y],axis = 1).corr()
#print(pd.concat([X_clean,y],axis = 1))
print(correlation_table.SalePrice[abs(correlation_table.SalePrice) >= 0.1])
high_correlation_features = list(correlation_table.SalePrice[abs(correlation_table.SalePrice) >= 0.1].index)
high_correlation_features.remove("SalePrice")
print(high_correlation_features)
#snb.heatmap(correlation_table,annot=True)
#ptl.show()

LotFrontage     0.334820
LotArea         0.263843
OverallQual     0.790982
YearBuilt       0.522897
YearRemodAdd    0.507101
                  ...   
227            -0.242598
228            -0.118069
232            -0.153990
233             0.352060
SalePrice       1.000000
Name: SalePrice, Length: 125, dtype: float64
['LotFrontage', 'LotArea', 'OverallQual', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'BsmtFullBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', 'ScreenPorch', 0, 3, 4, 7, 8, 10, 11, 12, 18, 27, 28, 32, 34, 35, 37, 40, 41, 42, 44, 46, 47, 48, 50, 51, 52, 67, 69, 72, 77, 81, 83, 87, 93, 94, 99, 102, 106, 107, 114, 117, 122, 123, 126, 127, 128, 129, 130, 131, 132, 134, 137, 138, 139, 140, 141, 144, 145, 146, 147, 148, 152, 153, 155, 156, 157, 158, 160, 161, 174, 175, 

In [4]:
#Hyper Parameter Selection

from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

parameter_grid = {"n_estimators" : [200,500,700,900,1100],
                "random_state" : [0],
                "booster": ["gbtree","gblinear","dart"],
                "max_depth":[4,9]}

gstest = GridSearchCV(estimator = XGBRegressor(),param_grid = parameter_grid,cv=5)
gstest.fit(X_clean,y)
print("the best is")
print(gstest.best_params_)

[10:28:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { max_depth } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:28:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { max_depth } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[10:28:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { max_depth } might not be used.

  This may not be accurate due to some parameters are only used in language bi

In [35]:
#XGBoost model testing

from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

X_train, X_valid, y_train, y_valid = train_test_split(X_clean[high_correlation_features],y,test_size=.2,random_state=1,train_size=.8)

my_model_2 = XGBRegressor(n_estimators=200,random_state=0,booster = "gbtree",max_depth =4) # Your code here


# Fit the model
my_model_2.fit(X_train,y_train, early_stopping_rounds=10,eval_set=[(X_valid,y_valid)],verbose=False) # Your code here

# Get predictions
predictions_2 = my_model_2.predict(X_valid[high_correlation_features]) # Your code here

# Calculate MAE
mae_2 = mean_absolute_error(predictions_2,y_valid) # Your code here

# Uncomment to print MAE
print("Mean Absolute Error:" , mae_2)

pred_final = my_model_2.predict(X_test_clean[high_correlation_features])

Mean Absolute Error: 15857.644236943494


In [33]:
# Save test predictions to file

output = pd.DataFrame({'Id': X_test_clean.index,
                       'SalePrice': pred_final})
output.to_csv('submission.csv', index=False)

print("Done!!!")

Done!!!
